# LSTM parameter exploration
This notebook is a template for finding the LSTM model best suited for your needs

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import sys
parent_dir=os.path.dirname(os.getcwd())
sys.path.insert(0,parent_dir)
import rippl_AI
import aux_fcn

### Data download
4 uLED sessions will be downloaded: Amigo2 and Som2 will be used for training ; Dlx1 and Thy7 for validation


In [ ]:
from figshare.figshare.figshare import Figshare
fshare = Figshare()

article_ids = [16847521,16856137,14959449,14960085] 
sess=['Amigo2','Som2','Dlx1','Thy7']                                  
for id,s in zip(article_ids,sess):
    datapath = os.path.join(parent_dir,'Downloaded_data', f'{s}')
    if os.path.isdir(datapath):
        print(f"{s} session already exists. Moving on.")
    else:
        print("Downloading data... Please wait, this might take up some time")        # Can take up to 10 minutes
        fshare.retrieve_files_from_article(id,directory=datapath)
        print("Data downloaded!")

### Data load
The training sessions' LFP will be appended together in a list. The same will happen with the ripples detection times.
That is the required input for the training parser

In [ ]:
# The training sessions will be appended together. Replace this cell with your own data loading
train_LFPs=[]
train_GTs=[]
# Amigo2
path=os.path.join(parent_dir,'Downloaded_data','Amigo2','figshare_16847521')
LFP,GT=aux_fcn.load_lab_data(path)
train_LFPs.append(LFP)
train_GTs.append(GT)
# Som2
path=os.path.join(parent_dir,'Downloaded_data','Som2','figshare_16856137')
LFP,GT=aux_fcn.load_lab_data(path)
train_LFPs.append(LFP)
train_GTs.append(GT)

## Append all your validation sessions
val_LFPs=[]
val_GTs=[]
# Dlx1 Validation
path=os.path.join(parent_dir,'Downloaded_data','Dlx1','figshare_14959449')
LFP,GT=aux_fcn.load_lab_data(path)
val_LFPs.append(LFP)
val_GTs.append(GT)
# Thy07 Validation
path=os.path.join(parent_dir,'Downloaded_data','Thy7','figshare_14960085')
LFP,GT=aux_fcn.load_lab_data(path)
val_LFPs.append(LFP)
val_GTs.append(GT)

x_training,GT_training,x_val_list,GT_val_list=rippl_AI.prepare_training_data(train_LFPs,train_GTs,val_LFPs,val_GTs,sf=30000)

## LSTM training parameters

#### Parameters:
* Channels:  number of channels that will be used to train the model, extracted from the data shape defined in the previous cell
* Timesteps: number of samples that the will be processed at once
* Bidirectionality: if the model processes simutaneously the window forward and backwards
* Layers: number of LSTM layers
* Epoch: number of times the training data set is used to train the model
* Training batch: number of windows that are proccessed before weight updating

#
LSTM contains more parameters, feel free to add your own modifications. Check the oficial documentation:
https://keras.io/api/layers/recurrent_layers/lstm/

In [ ]:
conf=   {"timesteps":[15,40],       # 8,16,40 ...
        "bidirectional": [0],       # 0 or 1
        "layers": [1,2],            # 2,3,4
        "units": [10],              # 5,6,10,12...
        "epochs": [2],              # 1,2,3...
        "train_batch": [2**8]}      # 16,32,64 (Powers of two are recommended for computacional efficiency)

### Training

In [ ]:
# Desired sampling frequency of the models
sf=1250
th_arr=np.linspace(0.1,0.9,9)
model_name_arr=[]           # To plot in the next cell
model_arr=[]                # Actual model array, used in the next validation section
n_channels=x_training.shape[1]
timesteps_arr=conf['timesteps']

bi_arr=conf['bidirectional']
layer_arr=conf['layers']                                  
units_arr=conf['units']        
epochs_arr=conf['epochs']
train_batch_arr=conf['train_batch']   

l_ts=len(timesteps_arr)

l_bi=len(bi_arr)
l_layer =len(layer_arr)
l_units=len(units_arr)
l_epochs =len(epochs_arr)
l_batch =len(train_batch_arr)
n_iters=l_ts*l_bi*l_layer*l_units*l_epochs*l_batch
# GT is in the shape (n_events x 2), a y output signal with the same length as x is required
perf_train_arr=np.zeros(shape=(n_iters,len(th_arr),3)) # Performance array, (n_models x n_th x 3 ) [P R F1]
perf_test_arr=np.zeros_like(perf_train_arr)
timesteps_arr_ploting=[]            # Array that will be used in the validation, to be able to call the function predict

print(f'{n_channels} channels will be used to train the LSTM models')

print(f'{n_iters} models will be trained')

x_test_or,GT_test,x_train_or,GT_train=aux_fcn.split_data(x_training,GT_training,split=0.7,sf=sf)

y_test_or= np.zeros(shape=(len(x_test_or)))
for ev in GT_test:
    y_test_or[int(sf*ev[0]):int(sf*ev[1])]=1
y_train_or= np.zeros(shape=(len(x_train_or)))
for ev in GT_train:
    y_train_or[int(sf*ev[0]):int(sf*ev[1])]=1


for i_ts,timesteps in enumerate(timesteps_arr):

        x_train=x_train_or[:len(x_train_or)-len(x_train_or)%timesteps].reshape(-1,timesteps,n_channels)
        y_train=y_train_or[:len(y_train_or)-len(y_train_or)%timesteps].reshape(-1,timesteps,1)
        
        x_test=x_test_or[:len(x_test_or)-len(x_test_or)%timesteps].reshape(-1,timesteps,n_channels)
        y_test=y_test_or[:len(y_test_or)-len(y_test_or)%timesteps].reshape(-1,timesteps,1)

        for i_bi,bi in enumerate(bi_arr):
                for i_layer,layers in enumerate(layer_arr):
                    for i_units,units in enumerate(units_arr):
                        for i_epochs,epochs in enumerate(epochs_arr):
                            for i_batch,train_batch in enumerate(train_batch_arr):
                                iter=((((i_ts*l_bi+i_bi)*l_layer+i_layer)*l_units+ i_units)*l_epochs + i_epochs)*l_batch + i_batch
                                print(f"\nIteration {iter+1} out of {n_iters}")
                                print(f'Number of channels: {n_channels:d}, Time steps: {timesteps:d}, Bidirectional: {bi:d}\nN of layers: {layers:d}, N of units: {units:d}, epochs: {epochs:d}, Samples per batch: {train_batch:d}')

                                model=aux_fcn.build_LSTM(input_shape=(timesteps,n_channels),n_layers=layers,layer_size=units,bidirectional=bi)
                                # Training
                                model.fit(x_train, y_train, epochs=epochs,batch_size=train_batch,validation_data=(x_test,y_test), verbose=1)
                                model_arr.append(model)

                                # Prediction
                                y_test_predict = model.predict(x_test,verbose=1).reshape(-1,1,1)
                                y_train_predict= model.predict(x_train,verbose=1).reshape(-1,1,1)
    
                                for i,th in enumerate(th_arr):
                                    # Test
                                    ytest_pred_ind=aux_fcn.get_predictions_index(y_test_predict,th)/sf
                                    perf_test_arr[iter,i]=aux_fcn.get_performance(ytest_pred_ind,GT_test,0)[0:3]
                                    # Train
                                    ytrain_pred_ind=aux_fcn.get_predictions_index(y_train_predict,th)/sf
                                    perf_train_arr[iter,i]=aux_fcn.get_performance(ytrain_pred_ind,GT_train,0)[0:3]

                                # Saving the model
                                model_name=f"LSTM_Ch{n_channels:d}_Ts{timesteps:03d}_Bi{bi:d}_L{layers:d}_U{units:02d}_E{epochs:02d}_TB{train_batch:04d}"
                                model.save(os.path.join(parent_dir,'explore_models',model_name))

                                model_name_arr.append(model_name)
                                timesteps_arr_ploting.append(timesteps)

### Plot training results

In [ ]:
# Plot training results
fig,axs=plt.subplots(n_iters,2,figsize=(10,2*n_iters),sharey='col',sharex='col')

for i in range(n_iters):
    axs[i,0].plot(perf_train_arr[i,:,0],perf_train_arr[i,:,1],'k.-')
    axs[i,0].plot(perf_test_arr[i,:,0],perf_test_arr[i,:,1],'b.-')
    axs[i,1].plot(th_arr,perf_train_arr[i,:,2],'k.-')
    axs[i,1].plot(th_arr,perf_test_arr[i,:,2],'b.-')
    axs[i,0].set_title(model_name_arr[i])
    axs[i,0].set_ylabel('Precision')
    axs[i,1].set_ylabel('F1')
axs[-1,0].set_xlabel('Recall')
axs[-1,1].set_xlabel('Threshold')
axs[0,0].legend(['Training','Test'])
plt.show()

### Validation

In [ ]:
# For loop iterating over the models
fig,axs=plt.subplots(n_iters,2,figsize=(10,2*n_iters),sharey='col',sharex='col')
for n_m,model in enumerate(model_arr):
    F1_arr=np.zeros(shape=(len(x_val_list),len(th_arr))) #(n_val_sess x n_th) Array where the F1 val of each sesion will be stored
    for n_sess,LFP in enumerate(x_val_list):
        val_pred=rippl_AI.predict(LFP,sf=1250,arch='LSTM',new_model=model,n_channels=n_channels,n_timesteps=timesteps_arr_ploting[n_m])[0]
        for i,th in enumerate(th_arr):
            val_pred_ind=aux_fcn.get_predictions_index(val_pred,th)/sf
            F1_arr[n_sess,i]=aux_fcn.get_performance(val_pred_ind,GT_val_list[n_sess],verbose=False)[2]
    
    axs[n_m,0].plot(th_arr,perf_train_arr[n_m,:,2],'k.-')
    axs[n_m,0].plot(th_arr,perf_test_arr[n_m,:,2],'b.-')
    for F1 in F1_arr:
        axs[n_m,1].plot(th_arr,F1)
    axs[n_m,1].plot(th_arr,np.mean(F1_arr,axis=0),'k.-')
    axs[n_m,0].set_title(model_name_arr[n_m])
    axs[n_m,0].set_ylabel('Precision')
    axs[n_m,1].set_ylabel('F1')
axs[-1,0].set_xlabel('Recall')
axs[-1,1].set_xlabel('Threshold')
plt.show()
    